In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import re

df = pd.read_csv('unique_1000_characters_dataset.csv', sep='\t')
df.rename(columns={df.columns[0]: 'Character'}, inplace=True)
df['Character'] = df['Character'].apply(lambda x: re.sub(r'\s*#\d+', '', x))
trait_columns = ['BAP1', 'BAP2', 'BAP3', 'BAP4', 'BAP5']
df = df[['Character'] + trait_columns]

for col in trait_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df = df.dropna()

def get_category(name):
    name = name.lower()
    if 'cartoon' in name:
        return 'Cartoon'
    else:
        return 'Modern'

df['Category'] = df['Character'].apply(get_category)
df = df[df['Category'].isin(['Cartoon', 'Modern'])]
df.set_index('Character', inplace=True)

questions = {
    'BAP1': {
        "إيه أكتر حاجة بتحب تعملها؟": {
            "حل مسائل أو تفكير منطقي": 90,
            "ألعاب بسيطة أو حاجات مسلية": 40,
            "القراءة أو البرمجة أو التحليل": 80,
            "مش بفضل التفكير الكتير": 20
        }
    },
    'BAP2': {
        "بتتصرف إزاي وقت الضغط؟": {
            "بهدا وأحاول أركز": 90,
            "ببان هادي لكن ببقى مشدود": 70,
            "بتوتر بسرعة": 40,
            "بانفعل على طول": 20
        }
    },
    'BAP3': {
        "لما حد يستفزك، بتعمل إيه؟": {
            "بزعق فورًا": 90,
            "بتعصب جوايا": 70,
            "بمشي ومردش": 30,
            "بسكت ومش بعلق": 40
        }
    },
    'BAP4': {
        "لما حد يظلمك، رد فعلك؟": {
            "بواجهه بصراحة": 90,
            "بطنش": 30,
            "بطلب حد يساعدني": 60,
            "بستنى وقت مناسب وأتحرك": 70
        }
    },
    'BAP5': {
        "بتتعامل إزاي وسط صحابك؟": {
            "بهزر وأضحّكهم": 90,
            "ساكت وبضحك قليل": 50,
            "جدي في معظم الوقت": 30,
            "على حسب المود": 60
        }
    }
}

trait_names = {
    'BAP1': 'الذكاء / التحليل',
    'BAP2': 'الهدوء / التحكم',
    'BAP3': 'العصبية / الانفعال',
    'BAP4': 'الجرأة / الشجاعة',
    'BAP5': 'المرح / الاجتماعيّة'
}

def ask_questions():
    answers = {}
    for bap, qdata in questions.items():
        q_text = list(qdata.keys())[0]
        options = list(qdata[q_text].keys())
        print(f"\n{q_text}")
        for i, opt in enumerate(options):
            print(f"{i+1}. {opt}")
        while True:
            try:
                choice = int(input("اختيارك (رقم): "))
                if 1 <= choice <= len(options):
                    answers[bap] = qdata[q_text][options[choice-1]]
                    break
                else:
                    print("اختار رقم من الاختيارات.")
            except ValueError:
                print("اكتب رقم.")
    return answers

def find_top_characters(user_scores, top_n=3):
    user_vector = np.array([user_scores[f'BAP{i+1}'] for i in range(5)]).reshape(1, -1)
    char_vectors = df[trait_columns].values
    similarities = cosine_similarity(user_vector, char_vectors)[0]
    df['Similarity'] = similarities
    top_cartoon = df[df['Category'] == 'Cartoon'].nlargest(top_n, 'Similarity')
    top_modern = df[df['Category'] == 'Modern'].nlargest(top_n, 'Similarity')
    return top_cartoon, top_modern

def print_result(user_scores, top_cartoon, top_modern):
    def show_list(title, df_list):
        print(f"\n {title}:")
        if df_list.empty:
            print(" لا يوجد شخصيات مناسبة حالياً.")
        else:
            for i, (name, row) in enumerate(df_list.iterrows(), 1):
                print(f"  {i}. {name} بنسبة شبه {row['Similarity']*100:.1f}%")
    print("\n النتيجة:")
    show_list("الشخصيات الكرتونية الأقرب ليك", top_cartoon)
    show_list("الشخصيات العامة أو المشهورة الأقرب ليك", top_modern)
    print("\nتحليل سماتك:")
    for i in range(5):
        dim = f'BAP{i+1}'
        print(f"  {trait_names[dim]} → إنت: {user_scores[dim]}")

print("Who Are You Most Like ? - Cartoon & Celeb Edition")
user_scores = ask_questions()
top_cartoon, top_modern = find_top_characters(user_scores)
print_result(user_scores, top_cartoon, top_modern)


Who Are You Most Like ? - Cartoon & Celeb Edition

إيه أكتر حاجة بتحب تعملها؟
1. حل مسائل أو تفكير منطقي
2. ألعاب بسيطة أو حاجات مسلية
3. القراءة أو البرمجة أو التحليل
4. مش بفضل التفكير الكتير


اختيارك (رقم):  3



بتتصرف إزاي وقت الضغط؟
1. بهدا وأحاول أركز
2. ببان هادي لكن ببقى مشدود
3. بتوتر بسرعة
4. بانفعل على طول


اختيارك (رقم):  1



لما حد يستفزك، بتعمل إيه؟
1. بزعق فورًا
2. بتعصب جوايا
3. بمشي ومردش
4. بسكت ومش بعلق


...


لما حد يظلمك، رد فعلك؟
1. بواجهه بصراحة
2. بطنش
3. بطلب حد يساعدني
4. بستنى وقت مناسب وأتحرك


اختيارك (رقم):  1



بتتعامل إزاي وسط صحابك؟
1. بهزر وأضحّكهم
2. ساكت وبضحك قليل
3. جدي في معظم الوقت
4. على حسب المود


اختيارك (رقم):  2



 النتيجة:

 الشخصيات الكرتونية الأقرب ليك:
  1. Ben 10 (Cartoon) بنسبة شبه 99.7%
  2. Goku (Cartoon) بنسبة شبه 99.5%
  3. Elsa (Cartoon) بنسبة شبه 99.0%

 الشخصيات العامة أو المشهورة الأقرب ليك:
  1. Asser Yassin (Arab Public) بنسبة شبه 99.8%
  2. Newton (Scientists) بنسبة شبه 99.7%
  3. Angela Merkel (Politicians) بنسبة شبه 99.7%

تحليل سماتك:
  الذكاء / التحليل → إنت: 80
  الهدوء / التحكم → إنت: 90
  العصبية / الانفعال → إنت: 70
  الجرأة / الشجاعة → إنت: 90
  المرح / الاجتماعيّة → إنت: 50
